# TODO
- [] 有0.0
- [v] 標準差

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler


In [3]:
FILENAME = input("File Name (in Data/resampled))") # '20230706_2e+07'

In [4]:
rename_mark = { "220045" : "Heart Rate"                           , \
                "220210" : "Respiratory Rate"                     , \
                "220179" : "Non Invasive Blood Pressure systolic" , \
                "220180" : "Non Invasive Blood Pressure diastolic", \
                "220277" : "O2 saturation pulseoxymetry"          }

In [5]:
df = pd.read_csv(f"data/{FILENAME}", engine='python')
# df.rename(columns=rename_mark, inplace=True)

In [6]:
scale = StandardScaler()
out = pd.DataFrame(scale.fit_transform(df.drop(columns="label")), columns=df.drop(columns="label").keys())
out

,220045,220179,220180,220210,220277
0,0.290983,-0.685077,-0.210074,-0.103668,0.230894
1,0.526726,0.223144,-0.159274,-0.585009,0.230894
2,0.880339,0.128538,0.856723,0.016668,0.076039
3,0.330274,-0.344494,0.094726,0.738680,0.540603
4,-0.769858,-0.009966,0.153145,0.137003,0.385749
...,...,...,...,...,...
183,1.430405,-2.463677,-1.480070,1.821698,-1.085371
184,1.037501,1.585476,2.279119,2.303039,0.385749
185,-0.101921,1.188129,1.771121,0.016668,-0.001388
186,0.683887,0.790782,1.263122,-0.224003,0.463176


In [7]:
scale = StandardScaler()
df_spl, df_label_spl = df.drop(columns="label"), df["label"]
df_spl = pd.DataFrame(scale.fit_transform(df_spl), columns=df_spl.keys())
X_train, X_test, y_train, y_test = train_test_split(df_spl, df_label_spl, test_size=0.2, stratify=df_label_spl)

In [8]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
model = tree.DecisionTreeClassifier(max_depth=3)
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cm

array([[15,  4],
       [ 6, 13]])

In [9]:
y_test.size

38

In [14]:
from graphviz import Source
features = df.drop(columns="label").keys()
dot_data = tree.export_graphviz(model, feature_names=features)
graph = Source(dot_data)
graph.render(view=True, format="pdf", filename=f"../Charts/tree{FILENAME}")

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
len(X_train), len(X_test)

(150, 38)

In [ ]:
cross_val_score(model, X_test, y_test, cv=5, scoring='accuracy').mean()

0.55

In [ ]:
model.score(X_test, y_test)

0.6578947368421053

In [ ]:
scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=3), df_under_resample, df_under_label_resample, cv=5, scoring='accuracy')
print([round(i, 2) for i in scores])
print(round(scores.mean(), 2))

NameError: name 'df_under_resample' is not defined